**TITRE**

In [7]:
import math

class AbstractRule:
    
    def _set_grammar(self,gram):
        self._grammar = gram
        
    def valuation(self):
        raise NotImplementedError
        
    def count(self):
        raise NotImplementedError

        
#Exception triggered when a rule uses a literal not in the grammar
class UnknownLiteralError(Exception):
    
    def __init__(self,literal):
        self.literal = literal

        
class ConstructorRule(AbstractRule):
    
    def __init__(self, *args):
        self._parameters = args
        #print (self._parameters[0])
        #print (self._parameters[1])
        self._valuation = math.inf

    def valuation(self):
        return self._valuation
    
    #The Grammar is supposed non ambiguous
    #check if all literals used in the rule are parts of the grammar
    def _verif_rule(self):
        try:
            self._grammar[self._parameters[0]]
        except KeyError:
            raise UnknownLiteralError(self._parameters[0])
        try:
            self._grammar[self._parameters[1]]
        except KeyError:
            raise UnknownLiteralError(self._parameters[1])

    #Return true if there was no update
    def _update_valuation(self):
            #print("Valuation : Constructor")
            #print("old :"+str(self._valuation))
            self._old_val = self._valuation
            self._valuation = self._calc_valuation()
            #print("new :"+str(self._valuation))
            #print(self._old_val == self._valuation)
            return (self._old_val == self._valuation)

        
class UnionRule(ConstructorRule):
    
    def __init__(self,fst,snd):
        #print ("union")
        super().__init__(fst,snd)
        
    def _calc_valuation(self):
        #print ("union")
        return min(self._grammar[self._parameters[0]].valuation(),
                   self._grammar[self._parameters[1]].valuation())
    
    def count(self, i):
        return self._grammar[self._parameters[0]].count(i) + self._grammar[self._parameters[1]].count(i)
    
        
class ProductRule(ConstructorRule):
    
    def __init__(self,fst,snd,cons):
        #print("product")
        super().__init__(fst,snd)
        self._constructor = cons
        
    def _calc_valuation(self):
        #print("product")
        return (self._grammar[self._parameters[0]].valuation() +
               self._grammar[self._parameters[1]].valuation())
    
    def count(self, i):
        res = 0
        valN1 = self._grammar[self._parameters[0]].valuation()
        valN2 = self._grammar[self._parameters[1]].valuation()
        for k in range (valN1, i-valN2):
            res += self._grammar[self._parameters[0]].count(k) * self._grammar[self._parameters[1]].count(i-k)

    
class ConstantRule(AbstractRule):
    
    def __init__(self,obj):
        self._object = obj
        
    #We chose to put _update_valuation here too
    #to avoid testing the presence of the function
    #every time we want to call it on a rule
    def _update_valuation(self):
        return True

    
class EpsilonRule(ConstantRule):
    
    def __init__(self,obj):
        super().__init__(obj)
    
    def valuation(self):
        return 0
    
    def count(self, i):
        if i != 0:
            return 0
        else :
            return 1
    
    
class SingletonRule(ConstantRule):
    
    def __init__(self,obj):
        super().__init__(obj)
        
    def valuation(self):
        return 1
    
    def count(self, i):
        if i != 1:
            return 0
        else :
            return 1

#Triggered when a non terminal does not generate anything
class CircularGrammarError(Exception):
    
    def __init__(self,grammar,name):
        self.name = name
        self.grammar = grammar
        
    
def init_grammar(gram):
    #set the grammar for all rules
    for rule in gram.values() :
        rule._set_grammar(gram)
        #verifies every non-constant rule (check if all literal used in the rules are in the grammar)
        if isinstance(rule, ConstructorRule):
            rule._verif_rule()

    #As long as there's a change we update again
    #(we chose to make _update_valuation available
    #to any rule)
    while not all(rule._update_valuation() for rule in gram.values()):
        pass
    for name,rule in gram.items() :
        if rule.valuation() == math.inf :
            raise CircularGrammarError(gram,name)

In [13]:
#Grammar definition for binary trees
treeGram = {"Tree" : UnionRule("Node","Leaf"),
            "Node" : ProductRule("Tree","Tree",
                                lambda a, b : Node(a, b)),
            "Leaf" : SingletonRule("Leaf")}
init_grammar(treeGram)

In [19]:
#tests for the binary trees grammar

#valuation

assert (treeGram["Tree"].valuation() == 1)
assert (treeGram["Node"].valuation() == 2)
assert (treeGram["Leaf"].valuation() == 1)

In [12]:
#Grammar defintion for Fibonacci words
fiboGram = {"Fib" : UnionRule("Vide", "Cas1"),
            "Cas1" : UnionRule("CasAu", "Cas2"),
            "Cas2" : UnionRule("AtomB", "CasBAu"),
            "Vide" : EpsilonRule(""),
            "CasAu" : ProductRule("AtomA", "Fib","".join),
            "AtomA" : SingletonRule("A"),
            "AtomB" : SingletonRule("B"),
            "CasBAu" : ProductRule("AtomB", "CasAu","".join)}
init_grammar(fiboGram)

In [21]:
#tests for the fibonacci words grammar

#valuation

assert (fiboGram["Fib"].valuation() == 0)
assert (fiboGram["Cas1"].valuation() == 1)
assert (fiboGram["Cas2"].valuation() == 1)
assert (fiboGram["Vide"].valuation() == 0)
assert (fiboGram["CasAu"].valuation() == 1)
assert (fiboGram["AtomA"].valuation() == 1)
assert (fiboGram["AtomB"].valuation() == 1)
assert (fiboGram["CasBAu"].valuation() == 2)

In [26]:
#Grammar definition for all the words with A and B letters
abGram = {"Words" : UnionRule("Vide","Cas1"),
          "Cas1" : UnionRule("CasAu","CasBu"),
          "AtomA" : SingletonRule("A"),
          "AtomB" : SingletonRule("B"),
          "CasAu" : ProductRule("AtomA", "Words","".join),
          "CasBu" : ProductRule("AtomB", "Words", "".join),
          "Vide" : EpsilonRule(""),
          }
init_grammar(abGram)

In [27]:
# tests for the (a,b) words grammar

#valuation
#TODO


In [29]:
#Grammar definition for Dyck words
dyckGram = {"Dyck" : UnionRule("Vide","Cas1"),
            "Cas1" : UnionRule("CasG","CasDouble"),
            "Vide" : EpsilonRule(""),
            "AtomG" : SingletonRule("("),
            "AtomD" : SingletonRule(")"),
            "CasG" : ProductRule("AtomG","CasD", "".join),
            "CasD" : ProductRule("Dyck","AtomD", "".join),
            "CasDouble" : ProductRule("Dyck","Dyck", "".join),
            }
init_grammar(dyckGram)

In [31]:
# tests for the Dyck words grammar

#valuation

assert (dyckGram["Dyck"].valuation() == 0)
assert (dyckGram["Cas1"].valuation() == 0)
assert (dyckGram["Vide"].valuation() == 0)
assert (dyckGram["AtomG"].valuation() == 1)
assert (dyckGram["AtomD"].valuation() == 1)
assert (dyckGram["CasG"].valuation() == 2)
assert (dyckGram["CasD"].valuation() == 1)
assert (dyckGram["CasDouble"].valuation() == 0)

In [ ]:
#Grammar definition for the (A,B) words without 3 times the same letter in a row
ab3Gram = {"Words" : UnionRule("Vide","Cas1"),
          "Cas1" : UnionRule("CasAu","CasBu"),
          "AtomA" : SingletonRule("A"),
          "AtomB" : SingletonRule("B"),
          "CasAA" : ProductRule("AtomB","CasAB"),
          "CasBB" : ProductRule("AtomB","AtomB"),
          "CasAB" : ProductRule("AtomA", "AtomB"),
          "CasBA" : ProductRule("AtomB", "AtomA"),
          "CasAu" : ProductRule("AtomA", "StartA","".join),
          "CasBu" : ProductRule("AtomB", "Words", "".join),
          "StartA" : Union()
          "Vide" : EpsilonRule(""),
          }
init_grammar(ab3Gram)

In [ ]:
# tests for the (A,B) words without 3 times the same letter in a row grammar

#valuation
